In [1]:
import numpy as np
import pandas as pd

In [2]:
# movies.csv
url = 'https://drive.google.com/file/d/1EknR8ceW4HEkjW-41F4z_IVku0RZERWH/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies = pd.read_csv(path)

# links.csv
url = 'https://drive.google.com/file/d/1F0jEbRxpqt1L6B_OjiLK29wZtj5JIocV/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
links = pd.read_csv(path)

# 'ratings.csv'
url = 'https://drive.google.com/file/d/1IdcUbfqlIIY0djoAtTKEDXhL6TCXyCoG/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings = pd.read_csv(path) 

# tags.csv
url = 'https://drive.google.com/file/d/1kOIkbYH6xjM_38EKqienvHtzmWUGunt4/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
tags = pd.read_csv(path)

1. explore dfs
2. rename coulmns
3. merge ratings and movies
4. drop rows with lesser number and timestamp
5. creating new dataframe with user count and average rating
6. calculate score
7. return the highest score


# CLEANING

In [3]:
ratings.drop(columns = "timestamp")

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [17]:
Ratings = ratings.rename({'userId ': 'userId', 'movieId ': 'movieId', 'rating ':'rating'}, axis=1) 

In [18]:
Movies = movies.rename({'movieId ': 'movieId', 'title                                                                                                                                                          ':'Title'
, 'genres' :'genres'}, axis=1) 

In [19]:
Movies['Title'] = Movies['Title'].str.strip()

In [7]:
#Movies.set_index("title", inplace = True)

In [20]:
Movies.columns

Index(['movieId', 'Title', 'genres'], dtype='object')

# CREATE FUNCTION: POPULARITY INDEX

In [21]:
def popular_movie_list(N):
  # MERGING
  Ratings_Movies = Ratings.merge(Movies, on='movieId')

  # DROPPING the less popular movies\
  Ratings_drop = Ratings_Movies["movieId"].value_counts()
  to_remove = Ratings_drop[Ratings_drop <= 10].index
  Ratings_Movies_new = Ratings_Movies[~Ratings_Movies.movieId.isin(to_remove)]

  # CREATING DF CONTAINS: USER COUNT AND AVG RATING
  Test_df = pd.DataFrame(Ratings_Movies_new.groupby(["movieId"]).userId.count())
  Test_df["average_rating"] = Ratings_Movies_new.groupby(["movieId"]).rating.mean()
  popular_movies = Test_df.merge(Movies, on='movieId')

  #indexing
  popular_movies.set_index('Title', inplace = True)

  # Calculating the Score
  popular_movies["score"] = round(popular_movies["userId"]*popular_movies["average_rating"] / (popular_movies["userId"] + popular_movies["average_rating"] ),2)
  return popular_movies.score.sort_values(ascending = False).head(N).index

In [22]:
popular_movie_list(4)

Index(['Shawshank Redemption, The (1994)', 'Godfather, The (1972)',
       'Fight Club (1999)', 'Star Wars: Episode IV - A New Hope (1977)'],
      dtype='object', name='Title')